In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from IPython.core.display import display, HTML
from pyspark.rdd import *
from pyspark.sql.column import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

def printDf(SparkDF):
    newdf = SparkDF.toPandas()
    return HTML(newdf.to_html())

spark = SparkSession.builder.appName("Operations").getOrCreate()

use_col_X = pd.read_csv("../config/use_col_X_202012.csv")
use_col_y = pd.read_csv("../config/use_col_y_202012.csv")

cat_cols = use_col_X[use_col_X["type"] == "category"].ja.tolist()
date_cols = use_col_X[use_col_X["type"] == "date"].ja.tolist()
obj_cols = use_col_X[use_col_X["type"] == "object"].ja.tolist()
int_cols = use_col_X[use_col_X["type"] == "int64"].ja.tolist()
float_cols = use_col_X[use_col_X["type"] == "float64"].ja.tolist()

read_dtype = {}
for col in use_col_X.ja.tolist():
    if col in cat_cols:
        read_dtype[col] = 'category'
    elif col in date_cols:
        read_dtype[col] = 'str'
    elif col in obj_cols:
        read_dtype[col] = 'object'
    elif col in int_cols:
        read_dtype[col] = 'int64'
    else:
        read_dtype[col] = 'float64'
        
struct_type = []
for col in use_col_X.ja.tolist():
    if read_dtype[col] == 'category':
        struct_type.append(StructField(col, IntegerType(), False))
    elif read_dtype[col] == "str":
        struct_type.append(StructField(col, StringType(), False))
    elif read_dtype[col] == "object":
        struct_type.append(StructField(col, IntegerType(), False))
    elif read_dtype[col] == "int64":
        struct_type.append(StructField(col, IntegerType(), False))
    elif read_dtype[col] == "float64":
        struct_type.append(StructField(col, DoubleType(), False))

schema = StructType(struct_type)

df_X = spark.read\
  .format("csv")\
  .options(header="true")\
  .load('../data/feature_202012.csv')#, schema=schema)
    
df_y = spark.read.csv("../data/label_202012.csv", header=True)
df_y = df_y.select(use_col_y["ja"].tolist())

df_X = df_X.select([c for c in use_col_X["ja"].tolist()])
df_y = df_y.select([c for c in use_col_y["ja"].tolist()])

df_X_en = df_X.toDF(*use_col_X["en"].tolist())
df_y_en = df_y.toDF(*use_col_y["en"].tolist())
df_y = df_y_en.select("service_id", "_IS_A_NPS_DETRACTOR")

df_X = df_X_en.drop(*use_col_X[use_col_X["type"] == "date"].en.tolist())

display(printDf(df_X.limit(5)))

,service_id,register_date,CAT_register_date,contract_months,contract_years,CAT_contract_years,contractor_gender,contractor_age,user_gender,user_age,billing_group,payment_methods,item_current,CAT_item_current,CAT_item_current_detail,use_days_from_change_to_now,purchase_time_S_item_current,purchase_time_K_item_current,channel_item_current,CAT_channel_current,primary_agency_item_current,agency_item_current,item_last,CAT_item_last,CAT_item_last_detail,channel_item_last,CAT_channel_item_last,primary_agency_item_last,agency_item_last,item_before_2times,CAT_item_before_2times,CAT_item_before_2times_detail,channel_item_before_2times,CAT_channel_item_before_2times,primary_agency_item_before_2times,agency_item_before_2times,plan_current,CAT_plan_current,CAT_plan_current_place,plan_last,CAT_plan_last,CAT_plan_last_place,data_plan_current,CAT_data_plan_current,CAT_data_plan_current_place,data_plan_last,CAT_data_plan_last,CAT_data_plan_last_place,join_place_basicpack,join_place_anshin,flg_basicpack,flg_anshin,BB_type,CAT_BB_type,CAT_BB_and_phone,flg_link_YahooID,type_long_privilege,flg_long_privilege,smartphone_num_in_FID,plan50GB_num_in_FID,smartphone_num_wo_me_in_FID,plan50GB_num_wo_me_in_FID,smartphone_num_w/o_me_in_FID,50GBplan_num_w/o_me_in_FID,CAT_plan50GB_num_wo_me_in_FID,CAT_smartphone_num_w/o_me_in_FID,CAT_50GBplan_num_w/o_me_in_FID,YL_count_5M,YL_count_4M,YL_count_3M,YL_count_2M,YL_count_1M,YL_count_AVG,CAT_YL_count_AVG_S,CAT_YL_count_AVG_K,YL_payment_5M,YL_payment_4M,YL_payment_3M,YL_payment_2M,YL_payment_1M,YL_payment_AVG,CAT_YL_price_AVG,mysb,raiten,kaden,web_cnt_1m,web_time_1m,shop_cnt_1m,call_cnt_1m,web_cnt_2m,web_time_2m,shop_cnt_2m,call_cnt_2m,web_cnt_3m,web_time_3m,shop_cnt_3m,call_cnt_3m,web_top_cnt_1m,web_data_cnt_1m,web_inquiry_cnt_1m,web_point_cnt_1m,web_change_cnt_1m,web_payec_cnt_1m,web_opt_plan_cnt_1m,web_other_cnt_1m,web_top_time_1m,web_data_time_1m,web_inquiry_time_1m,web_point_time_1m,web_change_time_1m,web_payec_time_1m,web_opt_plan_time_1m,web_other_time_1m,shop_annai_cnt_1m,shop_shinki_cnt_1m,shop_kihen_cnt_1m,shop_zyohen_cnt_1m,web_top_cnt_2m,web_data_cnt_2m,web_inquiry_cnt_2m,web_point_cnt_2m,web_change_cnt_2m,web_payec_cnt_2m,web_opt_plan_cnt_2m,web_other_cnt_2m,web_top_time_2m,web_data_time_2m,web_inquiry_time_2m,web_point_time_2m,web_change_time_2m,web_payec_time_2m,web_opt_plan_time_2m,web_other_time_2m,shop_annai_cnt_2m,shop_shinki_cnt_2m,shop_kihen_cnt_2m,shop_zyohen_cnt_2m,web_top_cnt_3m,web_data_cnt_3m,web_inquiry_cnt_3m,web_point_cnt_3m,web_change_cnt_3m,web_payec_cnt_3m,web_opt_plan_cnt_3m,web_other_cnt_3m,web_top_time_3m,web_data_time_3m,web_inquiry_time_3m,web_point_time_3m,web_change_time_3m,web_payec_time_3m,web_opt_plan_time_3m,web_other_time_3m,shop_annai_cnt_3m,shop_shinki_cnt_3m,shop_kihen_cnt_3m,shop_zyohen_cnt_3m,flg_paypay,beginning_date_paypay,channel_paypay,paypay_cnt_5m,paypay_cnt_4m,paypay_cnt_3m,paypay_cnt_2m,paypay_cnt_1m,paypay_cnt_AVG,CAT_paypay_cnt_AVG,paypay_payment_5m,paypay_payment_4m,paypay_payment_3m,paypay_payment_2m,paypay_payment_1m,paypay_payment_AVG,CAT_paypay_payment_AVG,paypay_cb_5m,paypay_cb_4m,paypay_cb_3m,paypay_cb_2m,paypay_cb_1m,paypay_cb_AVG,CAT_paypay_cb_AVG
0,100005398,199611,1,288,25年目,12,男性,58,男性,58,30,クレジットカード,iPhone SE_2 64GB,1,13,3,3,1,代理店ショップ,1,株式会社コスモネット,ソフトバンク北広島,iPhoneSE_16GB,1,1,代理店ショップ,1,株式会社ラネット,ソフトバンクインターヴィレッジ大曲,109SH,3,8,代理店ショップ,1,株式会社エム・タス,ソフトバンク北広島,基本プラン（音声）,8,1,通話定額ライト基本料,2,1,データプランメリハリ,12,1,データ定額 ５ＧＢ（ｉ）,3,1,2,1,2,1,SoftBank 光,1,2,1,1,1,3,2,2,2,1,1,3,2,2,0,3,0,0,0,0,1,1,0,41710,0,0,0,0,1,1,1,2,0,0,0,0,0,0,0,0,1,1.506,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1.506,0,0,0,0,0,0,0,2,0,3,0,1,202004,セルフ,2,2,0,0,0,0,1,996,15370,0,0,0,0,1,4,153,0,0,0,0,1
1,100006991,199611,1,287,24年目,12,女性,46,女性,46,30,クレジットカード,iPhone SE_2 128GB,1,13,2,2,1,代理店ショップ,1,コネクシオ株式会社,ソフトバンク松戸西口,iPhone 6s 64GB,1,3,代理店ショップ,1,株式会社ベルパーク,【継続用】ソフトバンク竹ノ塚,iPhone 5 32GB,1,1,代理店ショップ,1,株式会社アレン,ソフトバンク竹ノ塚,基本プラン（音声）,8,1,通話定額基本料（４Ｇ

## ラベルエンコーディング

In [2]:
def labelEncode(df, labelDict):
    # encode labels with value between 0 and n_classes-1
    # if labelDict is passed, encode label in same way as first.
    if labelDict == None:
        labelDict = {}
    cols = df.columns
    for i in df.schema:
        colName = i.name
        nColName = colName + "-encoded"
        colType = i.dataType
        if isinstance(colType, StringType):
            if colName in labelDict:
                eItems = labelDict[colName]
            else:
                eItems = {}
                for (x, i) in enumerate(df.groupBy(colName).agg({colName: "count"}).select(colName).collect()):
                    eItems[i[0]] = x
                labelDict[colName] = eItems
                #print(colName, eItems)
            eItems = spark.sparkContext.broadcast(eItems)
            def colLabelEncoder(a):
                topic = eItems.value
                if a in topic:
                    return topic[a]
                else:
                    sys.stderr.write("unencoded value found: "+str(a)+". assigned -1.\n")
                    return -1
            udfColLabelEncoder = udf(colLabelEncoder, IntegerType())
            df2 = df.withColumn(nColName, udfColLabelEncoder(df[colName])).drop(colName).withColumnRenamed(nColName, colName)
            df = df2.select(cols)
        else:
            pass
    return (df, labelDict)

In [3]:
(df_X2, labelDict_X) = labelEncode(df_X, None)
(df_y2, labelDict_y) = labelEncode(df_y, None)
display(printDf(df_X2.limit(10)))

,service_id,register_date,CAT_register_date,contract_months,contract_years,CAT_contract_years,contractor_gender,contractor_age,user_gender,user_age,billing_group,payment_methods,item_current,CAT_item_current,CAT_item_current_detail,use_days_from_change_to_now,purchase_time_S_item_current,purchase_time_K_item_current,channel_item_current,CAT_channel_current,primary_agency_item_current,agency_item_current,item_last,CAT_item_last,CAT_item_last_detail,channel_item_last,CAT_channel_item_last,primary_agency_item_last,agency_item_last,item_before_2times,CAT_item_before_2times,CAT_item_before_2times_detail,channel_item_before_2times,CAT_channel_item_before_2times,primary_agency_item_before_2times,agency_item_before_2times,plan_current,CAT_plan_current,CAT_plan_current_place,plan_last,CAT_plan_last,CAT_plan_last_place,data_plan_current,CAT_data_plan_current,CAT_data_plan_current_place,data_plan_last,CAT_data_plan_last,CAT_data_plan_last_place,join_place_basicpack,join_place_anshin,flg_basicpack,flg_anshin,BB_type,CAT_BB_type,CAT_BB_and_phone,flg_link_YahooID,type_long_privilege,flg_long_privilege,smartphone_num_in_FID,plan50GB_num_in_FID,smartphone_num_wo_me_in_FID,plan50GB_num_wo_me_in_FID,smartphone_num_w/o_me_in_FID,50GBplan_num_w/o_me_in_FID,CAT_plan50GB_num_wo_me_in_FID,CAT_smartphone_num_w/o_me_in_FID,CAT_50GBplan_num_w/o_me_in_FID,YL_count_5M,YL_count_4M,YL_count_3M,YL_count_2M,YL_count_1M,YL_count_AVG,CAT_YL_count_AVG_S,CAT_YL_count_AVG_K,YL_payment_5M,YL_payment_4M,YL_payment_3M,YL_payment_2M,YL_payment_1M,YL_payment_AVG,CAT_YL_price_AVG,mysb,raiten,kaden,web_cnt_1m,web_time_1m,shop_cnt_1m,call_cnt_1m,web_cnt_2m,web_time_2m,shop_cnt_2m,call_cnt_2m,web_cnt_3m,web_time_3m,shop_cnt_3m,call_cnt_3m,web_top_cnt_1m,web_data_cnt_1m,web_inquiry_cnt_1m,web_point_cnt_1m,web_change_cnt_1m,web_payec_cnt_1m,web_opt_plan_cnt_1m,web_other_cnt_1m,web_top_time_1m,web_data_time_1m,web_inquiry_time_1m,web_point_time_1m,web_change_time_1m,web_payec_time_1m,web_opt_plan_time_1m,web_other_time_1m,shop_annai_cnt_1m,shop_shinki_cnt_1m,shop_kihen_cnt_1m,shop_zyohen_cnt_1m,web_top_cnt_2m,web_data_cnt_2m,web_inquiry_cnt_2m,web_point_cnt_2m,web_change_cnt_2m,web_payec_cnt_2m,web_opt_plan_cnt_2m,web_other_cnt_2m,web_top_time_2m,web_data_time_2m,web_inquiry_time_2m,web_point_time_2m,web_change_time_2m,web_payec_time_2m,web_opt_plan_time_2m,web_other_time_2m,shop_annai_cnt_2m,shop_shinki_cnt_2m,shop_kihen_cnt_2m,shop_zyohen_cnt_2m,web_top_cnt_3m,web_data_cnt_3m,web_inquiry_cnt_3m,web_point_cnt_3m,web_change_cnt_3m,web_payec_cnt_3m,web_opt_plan_cnt_3m,web_other_cnt_3m,web_top_time_3m,web_data_time_3m,web_inquiry_time_3m,web_point_time_3m,web_change_time_3m,web_payec_time_3m,web_opt_plan_time_3m,web_other_time_3m,shop_annai_cnt_3m,shop_shinki_cnt_3m,shop_kihen_cnt_3m,shop_zyohen_cnt_3m,flg_paypay,beginning_date_paypay,channel_paypay,paypay_cnt_5m,paypay_cnt_4m,paypay_cnt_3m,paypay_cnt_2m,paypay_cnt_1m,paypay_cnt_AVG,CAT_paypay_cnt_AVG,paypay_payment_5m,paypay_payment_4m,paypay_payment_3m,paypay_payment_2m,paypay_payment_1m,paypay_payment_AVG,CAT_paypay_payment_AVG,paypay_cb_5m,paypay_cb_4m,paypay_cb_3m,paypay_cb_2m,paypay_cb_1m,paypay_cb_AVG,CAT_paypay_cb_AVG
0,81600,184,9,164,15,10,2,50,2,59,0,1,154,2,14,14,4,7,11,2,51,894,319,2,9,15,2,323,715,92,0,3,15,2,354,1195,7,2,1,38,9,2,27,10,2,90,2,2,4,2,1,0,1,1,3,0,1,1,1,10,10,9,7,7,0,4,4,29,14,26,24,0,39,7,1,2953,244,2718,2502,0,3705,9,0,0,1,27,4943,6,7,25,4312,6,8,92,18925,12,8,10,10,13,9,2,6,1,18,3623,1157,2818,479,657,384,8,2720,4,2,3,3,10,11,11,10,3,4,1,14,3185,1153,2475,416,524,401,7,2620,3,1,4,4,23,10,14,11,1,7,0,16,14339,1117,2399,462,519,389,5,2531,12,2,1,3,1,8,2,144,157,35,38,35,80,12,24518,11613,4953,5074,5097,7313,9,2632,1389,499,522,0,772,9
1,29846,184,9,37,10,10,0,31,0,38,0,1,132,2,14,115,29,7,11,2,31,3520,68,2,3,15,2,34,4012,89,2,8,15,2,188,998,7,2,1,31,7,4,27,10,2,90,2,4,2,2,1,0,1,1,3,0,1,1,6,7,7,3,3,3,2,2,2,29,26,26,24,0,39,7,1,2953,2667,2718,2502,0,3705,9,0,1,1,27,4943,6,7,25,4312,6,8,92,11657,8,8,1

## データマージ

In [4]:
df_X_y = df_X2.join(df_y2, ['service_id'],'inner')
#df_X_y.drop("service_id").columns
df_X_y.filter(df_X_y.user_age < 100)
df_X_y.filter(df_X_y.contractor_age<100)
display(printDf(df_X_y.limit(5)))

,service_id,register_date,CAT_register_date,contract_months,contract_years,CAT_contract_years,contractor_gender,contractor_age,user_gender,user_age,billing_group,payment_methods,item_current,CAT_item_current,CAT_item_current_detail,use_days_from_change_to_now,purchase_time_S_item_current,purchase_time_K_item_current,channel_item_current,CAT_channel_current,primary_agency_item_current,agency_item_current,item_last,CAT_item_last,CAT_item_last_detail,channel_item_last,CAT_channel_item_last,primary_agency_item_last,agency_item_last,item_before_2times,CAT_item_before_2times,CAT_item_before_2times_detail,channel_item_before_2times,CAT_channel_item_before_2times,primary_agency_item_before_2times,agency_item_before_2times,plan_current,CAT_plan_current,CAT_plan_current_place,plan_last,CAT_plan_last,CAT_plan_last_place,data_plan_current,CAT_data_plan_current,CAT_data_plan_current_place,data_plan_last,CAT_data_plan_last,CAT_data_plan_last_place,join_place_basicpack,join_place_anshin,flg_basicpack,flg_anshin,BB_type,CAT_BB_type,CAT_BB_and_phone,flg_link_YahooID,type_long_privilege,flg_long_privilege,smartphone_num_in_FID,plan50GB_num_in_FID,smartphone_num_wo_me_in_FID,plan50GB_num_wo_me_in_FID,smartphone_num_w/o_me_in_FID,50GBplan_num_w/o_me_in_FID,CAT_plan50GB_num_wo_me_in_FID,CAT_smartphone_num_w/o_me_in_FID,CAT_50GBplan_num_w/o_me_in_FID,YL_count_5M,YL_count_4M,YL_count_3M,YL_count_2M,YL_count_1M,YL_count_AVG,CAT_YL_count_AVG_S,CAT_YL_count_AVG_K,YL_payment_5M,YL_payment_4M,YL_payment_3M,YL_payment_2M,YL_payment_1M,YL_payment_AVG,CAT_YL_price_AVG,mysb,raiten,kaden,web_cnt_1m,web_time_1m,shop_cnt_1m,call_cnt_1m,web_cnt_2m,web_time_2m,shop_cnt_2m,call_cnt_2m,web_cnt_3m,web_time_3m,shop_cnt_3m,call_cnt_3m,web_top_cnt_1m,web_data_cnt_1m,web_inquiry_cnt_1m,web_point_cnt_1m,web_change_cnt_1m,web_payec_cnt_1m,web_opt_plan_cnt_1m,web_other_cnt_1m,web_top_time_1m,web_data_time_1m,web_inquiry_time_1m,web_point_time_1m,web_change_time_1m,web_payec_time_1m,web_opt_plan_time_1m,web_other_time_1m,shop_annai_cnt_1m,shop_shinki_cnt_1m,shop_kihen_cnt_1m,shop_zyohen_cnt_1m,web_top_cnt_2m,web_data_cnt_2m,web_inquiry_cnt_2m,web_point_cnt_2m,web_change_cnt_2m,web_payec_cnt_2m,web_opt_plan_cnt_2m,web_other_cnt_2m,web_top_time_2m,web_data_time_2m,web_inquiry_time_2m,web_point_time_2m,web_change_time_2m,web_payec_time_2m,web_opt_plan_time_2m,web_other_time_2m,shop_annai_cnt_2m,shop_shinki_cnt_2m,shop_kihen_cnt_2m,shop_zyohen_cnt_2m,web_top_cnt_3m,web_data_cnt_3m,web_inquiry_cnt_3m,web_point_cnt_3m,web_change_cnt_3m,web_payec_cnt_3m,web_opt_plan_cnt_3m,web_other_cnt_3m,web_top_time_3m,web_data_time_3m,web_inquiry_time_3m,web_point_time_3m,web_change_time_3m,web_payec_time_3m,web_opt_plan_time_3m,web_other_time_3m,shop_annai_cnt_3m,shop_shinki_cnt_3m,shop_kihen_cnt_3m,shop_zyohen_cnt_3m,flg_paypay,beginning_date_paypay,channel_paypay,paypay_cnt_5m,paypay_cnt_4m,paypay_cnt_3m,paypay_cnt_2m,paypay_cnt_1m,paypay_cnt_AVG,CAT_paypay_cnt_AVG,paypay_payment_5m,paypay_payment_4m,paypay_payment_3m,paypay_payment_2m,paypay_payment_1m,paypay_payment_AVG,CAT_paypay_payment_AVG,paypay_cb_5m,paypay_cb_4m,paypay_cb_3m,paypay_cb_2m,paypay_cb_1m,paypay_cb_AVG,CAT_paypay_cb_AVG,_IS_A_NPS_DETRACTOR
0,81600,184,9,164,15,10,2,50,2,59,0,1,154,2,14,14,4,7,11,2,51,894,319,2,9,15,2,323,715,92,0,3,15,2,354,1195,7,2,1,38,9,2,27,10,2,90,2,2,4,2,1,0,1,1,3,0,1,1,1,10,10,9,7,7,0,4,4,29,14,26,24,0,39,7,1,2953,244,2718,2502,0,3705,9,0,0,1,27,4943,6,7,25,4312,6,8,92,18925,12,8,10,10,13,9,2,6,1,18,3623,1157,2818,479,657,384,8,2720,4,2,3,3,10,11,11,10,3,4,1,14,3185,1153,2475,416,524,401,7,2620,3,1,4,4,23,10,14,11,1,7,0,16,14339,1117,2399,462,519,389,5,2531,12,2,1,3,1,8,2,144,157,35,38,35,80,12,24518,11613,4953,5074,5097,7313,9,2632,1389,499,522,0,772,9,0
1,29846,184,9,37,10,10,0,31,0,38,0,1,132,2,14,115,29,7,11,2,31,3520,68,2,3,15,2,34,4012,89,2,8,15,2,188,998,7,2,1,31,7,4,27,10,2,90,2,4,2,2,1,0,1,1,3,0,1,1,6,7,7,3,3,3,2,2,2,29,26,26,24,0,39,7,1,2953,2667,2718,2502,0,3705,9,0,1,1,27,4943,6,7,25,4

## Null削除

In [5]:
import pandas as pd
from pyspark.sql import SparkSession
from IPython.core.display import display, HTML
from pyspark.rdd import *
from pyspark.sql.column import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

def drop_null_columns(df):
    null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v>0]
    df = df.drop(*to_drop)
    return df

df_X_y2 = drop_null_columns(df_X_y)
df_X_y2 = df_X_y2.withColumn("_IS_A_NPS_DETRACTOR", df_X_y2["_IS_A_NPS_DETRACTOR"].cast(IntegerType()))

## 外れ値置換

In [6]:
#df2 = df2.replace(999.0, 0.0, "user_age")
#df2 = df2.replace(999.0, 0.0, "contractor_age")

## 特徴量追加

In [7]:
#df2 = df2.withColumn("can_vote", col('Age') >= 18)
#df2 = df2.withColumn("can_vote", col('Age') >= 18)

In [8]:
df_X_y2 = df_X_y2.sort('service_id').limit(100)

## Model with CV

In [ ]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.classification import GBTClassifier

print((df_X_y2.count(), len(df_X_y2.columns)))

target = '_IS_A_NPS_DETRACTOR'
featuresCols = df_X_y2.columns
featuresCols.remove('service_id')
featuresCols.remove(target)
# Concatenates all feature columns into a single feature vector in a new column "rawFeatures"
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
# Identifies categorical features and indexes them
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

gbt = GBTClassifier(labelCol=target)

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Define a grid of hyperparameters to test:
#  - maxDepth: max depth of each decision tree in the GBT ensemble
#  - maxIter: iterations, i.e., number of trees in each GBT ensemble
# In this example notebook, we keep these values small.  In practice, to get the highest accuracy, you would likely want to try deeper trees (10 or higher) and more trees in the ensemble (>100)
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 100])\
  .build()
# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol=gbt.getLabelCol())#, predictionCol=gbt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

train, test = df_X_y2.randomSplit([0.7, 0.3])
print("We have %d training examples and %d test examples." % (train.count(), test.count()))

pipelineModel = pipeline.fit(train)
predictions = pipelineModel.transform(test)

(100, 182)
We have 64 training examples and 36 test examples.


In [ ]:
pd_predictions = predictions.select("*").toPandas()

## Feature Importance

In [ ]:
pdf = pd.DataFrame({'Name': featureCols, 'Importance': pipelineModel.featureImportances.toArray()})
pdf2 = pdf.sort_values(by=['Importance'], ascending=False).reset_index().drop(['index'], axis=1)
display(pdf2.head(5))
f, ax = plt.subplots(figsize=(11,16))
sns.barplot(x=pdf2["Importance"], y=pdf2["Name"], orient='h')
ax.set_xlabel("feature inportance")
plt.show()